# Get Upcoming Event for Streamlit

In [ ]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from random import randint
import  random
import os
os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')

from cmath import nan
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Getting NEXT Fights & Scraping URLS

In [ ]:
def get_next_event_ufcstats():

    # scrape UFCStats website for next event

    url = 'http://www.ufcstats.com/statistics/events/upcoming'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('td', class_='b-statistics__table-col')
    event1_txt = soup.find('td', class_='b-statistics__table-col').text
    event_txt = event1_txt.replace('   ', '').replace('\n', '').strip()
    event_title = event_txt.split('  ')[0]
    event_date = event_txt.split('  ')[1]
    event1_url = event1.find('a')['href']
    data = pd.DataFrame({'event_title': [event_title], 'event_url': [event1_url], 'event_date': [event_date]})


    return data

In [ ]:
next_event = get_next_event_ufcstats()
next_event

#### Get all Data from Fighters in Next Event

In [ ]:
def get_fighter_urls(event_details_url):

    # scrape UFCStats website for next event
    # Returns a dataframe with fighter names, fight urls, and fight number

    page = requests.get(event_details_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click')
    n = 0
    next_event_data = pd.DataFrame()

    for event in events:
        fighters = events[n].find_all('p', class_='b-fight-details__table-text')
        fighter1 = fighters[0].text
        fighter1 = fighter1.replace('  ', '').replace('\n', '').strip()
        fighter2 = fighters[1].text
        fighter2 = fighter2.replace('  ', '').replace('\n', '').strip()
        fighter1_url = fighters[0].find('a')['href']
        fighter2_url = fighters[1].find('a')['href']
        next_event_data = next_event_data.append({'fighter1' :fighter1, 'fighter2:' : fighter2, 'fighter1_url': fighter1_url, 'fighter2_url':fighter2_url, 'fight#' : n+1}, ignore_index = True)
        n += 1

    return next_event_data

In [ ]:
fighter_urls = get_fighter_urls(next_event['event_url'].values[0])
fighter_urls

In [ ]:
def get_next_events_ufc_com(url):

    # make a looping version of get_next_events(url) 
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('div', class_='c-card-event--result__info')
    n = 0
    next_events_data = pd.DataFrame()

    for event in events:
        event_txt = events[n].text
        event_url = events[n].find('a')['href']
        event_url = 'https://www.ufc.com' + event_url
        event_title = event_txt.split('\n')[1]
        event_time = event_txt.split('/')[1]
        next_events_data = next_events_data.append({'event_title': event_title, 'event_url': event_url, 'event_date': event_time}, ignore_index=True)
        n += 1

    # only keep next event
    next_events_data = next_events_data.head(1)

    # add matchup
    main_fighter1 = next_events_data['event_title'].str.split('vs').str[0]
    main_fighter2 = next_events_data['event_title'].str.split('vs').str[1]

    next_events_data['main_fighter1'] = main_fighter1
    next_events_data['main_fighter2'] = main_fighter2
    return next_events_data

In [ ]:
next= get_next_events_ufc_com('https://www.ufc.com/events')
next

#### Get Next Events from UFCStats

In [ ]:
url = 'http://www.ufcstats.com/statistics/events/upcoming'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
    # get events
event = soup.findAll('i', class_ = 'b-statistics__table-content')
urls = soup.findAll('a', class_ = 'b-link b-link_style_black')
dates = soup.findAll('span', class_ = 'b-statistics__date')
n = 0
next_event_data = pd.DataFrame()

for i in event:
    # get event title, date, and url
    title = event[n].text
    title = title.replace('\n', '').strip()
    ti = title.split('  ')[0]
    date = dates[n].text.replace('\n', '').strip()
    #date = date.split('  ')[1]
    url = urls[n]['href']
    next_event_data = next_event_data.append({'event_title': ti, 'event_url': url, 'event_date': date}, ignore_index=True)
    n += 1


next_event_data['fighters'] = next_event_data['event_title'].str.split(': ').str[1]
next_event_data['main_fighter1'] = next_event_data['fighters'].str.split(' vs. ').str[0]
next_event_data['main_fighter2'] = next_event_data['fighters'].str.split(' vs. ').str[1]
next_event_data = next_event_data.head(1)
next_event_data


#### Get BestFightOdds Url

In [ ]:
def my_search_bfo(fighter1, fighter2):

    # Get urls for bestfightodds for events from GOOGLE, as it is more consistant than bfo search

    try:
        time.sleep(5)
        url = 'https://www.google.com/search?query=site:bestfightodds.com ' + fighter1 + ' ' + fighter2
        header = {
                    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
                    "X-Requested-With": "XMLHttpRequest"
    }
        r = requests.get(url, headers = header)
        # get all links
        soup = BeautifulSoup(r.text, 'html.parser')
        links = soup.find_all('a')
        # keep only events
        goog_response = [n['href'] for n in links if 'events' in n['href']]
        # clean up the link so it is only the url
        goog_response = [n.replace('/url?q=', '') for n in goog_response]
        # cut off the begining up until https
        goog_response = [n[n.find('https'):] for n in goog_response]
        # cut off the end after the &
        goog_response = [n[:n.find('&')] for n in goog_response]
        # keep only the first one
        goog_response = goog_response[0]
        return goog_response
    except:
        return np.nan


In [ ]:
# add the urls to the dataframe
next['bfo_url'] = next.apply(lambda x: my_search_bfo(x['main_fighter1'], x['main_fighter2']), axis=1)

In [ ]:
next

#### Add ufc_stats url

In [ ]:
next['ufcstats_url'] = next_event_data['event_url']
next['ufc_stats_event_title'] = next_event_data['event_title']
next['ufc_stats_date'] = next_event_data['event_date']

In [ ]:
next

In [ ]:
next.to_csv('data/final/events/upcoming_events_urls/' + str(next['event_title'].values[0]) + '.csv', index=False)

## Streamlit Functionality

We also need it to work in streamlit, without taking forever. So, there should be an updated fights_df with the NEXT fights added, which can then update the calculations on a consistant basis. 

In [ ]:
def get_next_events_ufc_com(url):

    # make a looping version of get_next_events(url) 

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('div', class_='c-card-event--result__info')
    n = 0
    next_events_data = pd.DataFrame()

    for event in events:
        event_txt = events[n].text
        event_url = events[n].find('a')['href']
        event_url = 'https://www.ufc.com' + event_url
        event_title = event_txt.split('\n')[1]
        event_time = event_txt.split('/')[1]
        next_events_data = next_events_data.append({'event_title': event_title, 'event_url': event_url, 'event_date': event_time}, ignore_index=True)
        n += 1

    # only keep top 6 events
    next_events_data = next_events_data.head(6)
    return next_events_data

In [ ]:
nel = get_next_events_ufc_com('https://www.ufc.com/events')
nel

In [ ]:
def get_fighter_urls(event_details_url):
    page = requests.get(event_details_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click')
    n = 0
    next_event_data = pd.DataFrame()

    for event in events:
        fighters = events[n].find_all('p', class_='b-fight-details__table-text')
        fighter1 = fighters[0].text
        fighter1 = fighter1.replace('  ', '').replace('\n', '').strip()
        fighter2 = fighters[1].text
        fighter2 = fighter2.replace('  ', '').replace('\n', '').strip()
        fighter1_url = fighters[0].find('a')['href']
        fighter2_url = fighters[1].find('a')['href']
        # get weight class
        weight_class = events[n].text
        wc = weight_class.find('Matchup')
        # delete everything from the end of word "Matchup" to the begining of the weight class
        weight_class = weight_class[wc:]

        weight_class = weight_class.replace('\n', '')
        weight_class = weight_class.replace('Matchup', '')
        weight_class = weight_class.replace('  ', '').strip()
        # delete everything other then the word with "weight" in it
        #weight_class = weight_class.split(' ')
        #weight_class = [n for n in weight_class if 'weight' in n]
        next_event_data = next_event_data.append({'fighter1' :fighter1, 'fighter2:' : fighter2, 'fighter1_url': fighter1_url, 
                                                    'fighter2_url':fighter2_url, 'fight#' : n+1, 'fight_weightclass' :weight_class}, ignore_index = True)
        n += 1

    return next_event_data


In [ ]:
# test
test = get_fighter_urls('http://ufcstats.com/event-details/56ec58954158966a')

In [ ]:
# Get events from ufcstats

url = 'http://www.ufcstats.com/statistics/events/upcoming'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
    # get events
event = soup.findAll('i', class_ = 'b-statistics__table-content')
urls = soup.findAll('a', class_ = 'b-link b-link_style_black')
dates = soup.findAll('span', class_ = 'b-statistics__date')
n = 0
next_event_data = pd.DataFrame()

for i in event:
    # get event title, date, and url
    title = event[n].text
    title = title.replace('\n', '').strip()
    ti = title.split('  ')[0]
    date = dates[n].text.replace('\n', '').strip()
    #date = date.split('  ')[1]
    url = urls[n]['href']
    next_event_data = next_event_data.append({'event_title': ti, 'event_url': url, 'event_date': date}, ignore_index=True)
    n += 1

next_event_data= next_event_data.head(3)
next_event_data

In [ ]:
fighter_urls = get_fighter_urls(next_event_data['event_url'].values[0])
fighter_urls

In [ ]:
# turn event_date to actual date
next_event_data['event_date'] = pd.to_datetime(next_event_data['event_date']).dt.date
event_date = next_event_data['event_date'].values[0]
event_date

In [ ]:
next_fight_df = fighter_urls[['fighter1', 'fighter2:', 'fight_weightclass']]
next_fight_df.columns = ['Fighter_A', 'Fighter_B', 'fight_weightclass']
next_fight_df['date'] = event_date
next_fight_df

#### Get Columns to Add Features



In [ ]:
# load final df to get columns from 

fights = pd.read_csv('data/final/aggregates/Double_Fights_DF_V14.csv') 

In [ ]:
fights.columns.to_list()

In [ ]:
fights.columns

### Add Features

- Start with Fighter Names
- Add odds data


In [ ]:
next_fight_df

In [ ]:
def get_odds(url):
    
        # to scrape odds from BFO
        # you can scrape by pretending to be a browser

        header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
        }
        r = requests.get(url, headers = header)
        dfs = pd.read_html(r.text)
        data = dfs[1]
        data.rename(columns={'Unnamed: 0': 'fighter'}, inplace=True)
        data['event_odds_url'] = url
        event_name = url.split('/')[4]
        data['event_id'] = event_name
        data.to_csv('data/final/events/odds_per_event/' + event_name + '.csv', index=False)
        return data

In [ ]:
event_odds = get_odds(next['bfo_url'].values[0])

In [ ]:
event_odds = event_odds[['fighter', 'DraftKings', '5D', 'Ref']]
event_odds

In [ ]:
# get rid of arrows in all columns
event_odds['DraftKings'] = event_odds['DraftKings'].str.replace('▲', '')
event_odds['DraftKings'] = event_odds['DraftKings'].str.replace('▼', '')
event_odds['5D'] = event_odds['5D'].str.replace('▲', '')
event_odds['5D'] = event_odds['5D'].str.replace('▼', '')
event_odds['Ref'] = event_odds['Ref'].str.replace('▲', '')
event_odds['Ref'] = event_odds['Ref'].str.replace('▼', '')

In [ ]:
# turn to numeric
event_odds['DraftKings'] = pd.to_numeric(event_odds['DraftKings'])
event_odds['5D'] = pd.to_numeric(event_odds['5D'])
event_odds['Ref'] = pd.to_numeric(event_odds['Ref'])

In [ ]:
def get_odds_for_fighter(fighter):
    try:
        data = event_odds[event_odds['fighter'] == fighter]
        dk_odds = data['DraftKings'].values[0]
        return dk_odds
    except:
        return np.nan


In [ ]:
next_fight_df['Fighter_A_Odds'] = next_fight_df['Fighter_A'].apply(get_odds_for_fighter)
next_fight_df['Fighter_B_Odds'] = next_fight_df['Fighter_B'].apply(get_odds_for_fighter)

In [ ]:
next_fight_df

#### Add Odds Change

In [ ]:
def get_bfodds(event_url):
    # scrape the changes in odds by event url
    
    driver = webdriver.Chrome()
    driver.get(event_url)

    # click show all
    driver.find_element_by_class_name('event-swing-expand').click()
    #wait 2 seconds
    time.sleep(2)

    event_swing_container = driver.find_element_by_id('event-swing-container')
    event_swing_container.text


    # the first half of the text is the odds changes, the seconds half are the names. Lets match them up. 
    # first split the text by new line
    event_swing_container.text.split('\n')

    # get the names. You can identify them because they will be more than 5 characters long
    names = [x for x in event_swing_container.text.split('\n') if len(x) > 5]
    names

    # get the odds changes
    odds_changes = [x for x in event_swing_container.text.split('\n') if '%' in x]
    odds_changes

    # get the event name
    event_name = driver.find_element_by_class_name('table-header')
    # name is h1 in event_name
    name = event_name.find_element_by_tag_name('h1')
    event = name.text

    # get the url
    url = driver.current_url


    # create a dataframe
    df = pd.DataFrame({'names': names, 'odds_changes': odds_changes, 'event': event, 'event_url': url})
    
    #close the driver
    driver.close()

    return df

In [ ]:
odds_changes = get_bfodds(next['bfo_url'].values[0])
odds_changes.head(2)

In [ ]:
# if ':' in odds_changes['odds_changes'].values[0]:
# split the odds changes by : and keep the second part
def fix_odds_changes(odds):
    if ':' in odds:
        oddz =  odds.split(':')[1].strip()
        # remove the % sign
        oddz = oddz.replace('%', '')
        oddz = float(oddz)
        return oddz
    else:
        oddz = odds.replace('%', '')
        oddz = float(oddz)
        return oddz

In [ ]:
def fix_names(name):
    if "Change since opening" in name:
        return name.split('Change since opening')[0].strip()[:-1]
    else:
        return name

In [ ]:
# fix the odds changes
odds_changes['odds_changes'] = odds_changes['odds_changes'].apply(fix_odds_changes)

In [ ]:
odds_changes['names'] = odds_changes['names'].apply(fix_names)

In [ ]:
odds_changes

In [ ]:
def get_odds_changes_for_fighter(fighter):
    try:
        data = odds_changes[odds_changes['names'] == fighter]
        odds_change = data['odds_changes'].values[0]
        return odds_change
    except:
        return np.nan

In [ ]:
next_fight_df['Fighter_A_Odds_Change'] = next_fight_df['Fighter_A'].apply(get_odds_changes_for_fighter)
next_fight_df['Fighter_B_Odds_Change'] = next_fight_df['Fighter_B'].apply(get_odds_changes_for_fighter)


In [ ]:
next_fight_df

### Add Distribution Stat Features

In [ ]:
# for each column in all_metric_cols, get the mean, std, etc. for each fighter
def get_fighter_running_dist_stats(fighter, date, col_to_get, stat_to_calc):
    data = fights[(fights['Fighter_A'] == fighter) | (fights['Fighter_B'] == fighter)]

    # only get fights before the date
    datey = pd.to_datetime(date)
    data['date'] = pd.to_datetime(data['date'])
    data = data[data['date'] < datey]

    # fighter could be either fighter A or fighter B
    fighter_data = pd.DataFrame()
    # when fighter is fighter A, get all fighter A data and copy it to fighter_data

    # fighterA df
    fighterA_df = fights[fights['Fighter_A'] == fighter]
    fighterB_df = fights[fights['Fighter_B'] == fighter]

    # keep only the fighters columns date, FighterA, and the col_to_get, 
    # do same for B, change col names from B to A, and concat
    fighterA_df = fighterA_df[['date', 'Fighter_A', 'A_' + col_to_get]]
    fighterA_df.rename(columns={'A_' + col_to_get: col_to_get, 'Fighter_A': 'fighter'}, inplace=True)
    fighterB_df = fighterB_df[['date', 'Fighter_B', 'B_' + col_to_get]]
    fighterB_df.rename(columns={'B_' + col_to_get: col_to_get, 'Fighter_B': 'fighter'}, inplace=True)

    # append the dataframes on fighter
    fighter_data = fighter_data.append(fighterA_df)
    fighter_data = fighter_data.append(fighterB_df)

    # get the chosen statistic
    if stat_to_calc == 'mean':
        x = fighter_data[col_to_get].mean()
    elif stat_to_calc == 'std':
        x = fighter_data[col_to_get].std()
    elif stat_to_calc == 'median':
        x = fighter_data[col_to_get].median()  
    return x

In [ ]:
# get non-specific column names for feature creation

dif_cols = [n for n in fights.columns if 'Dif_' in n]
A_cols = [n for n in fights.columns if n.startswith('A_')]
all_metric_cols = A_cols + dif_cols
A_cols2 = pd.DataFrame(A_cols)
A_cols2['nonspecific'] = A_cols2[0].str[2:]
the_cols = list(A_cols2['nonspecific'].unique())
the_cols = the_cols[:34]
the_cols

In [ ]:
for col in the_cols:
    for stat in ['mean', 'std', 'median']:
        next_fight_df['A_Rolling_' + col + '_' + stat] = next_fight_df.apply(lambda row: get_fighter_running_dist_stats(row['Fighter_A'], row['date'], col, stat), axis=1)
        next_fight_df['B_Rolling_' + col + '_' + stat] = next_fight_df.apply(lambda row: get_fighter_running_dist_stats(row['Fighter_B'], row['date'], col, stat), axis=1)

In [ ]:
fights

### Features: Top-Down Averages (Typical UFC Style)

In [ ]:
def get_top_down_averages(fighter, date, col_to_get, dataframe):

    # for each column in all_metric_cols, calculate the top-down averages, as opposed to bottom-up

    data = dataframe[(dataframe['Fighter_A'] == fighter) | (dataframe['Fighter_B'] == fighter)]

    # only get fights before the date

    datey = pd.to_datetime(date)
    data['date'] = pd.to_datetime(data['date'])
    data = data[data['date'] < datey]

    # fighter could be either fighter A or fighter B

    fighter_data = pd.DataFrame()

    # when fighter is fighter A, get all fighter A data and copy it to fighter_data

    # fighterA df

    fighterA_df = dataframe[dataframe['Fighter_A'] == fighter]
    fighterB_df = dataframe[dataframe['Fighter_B'] == fighter]
    
    # keep only the fighters columns date, FighterA, and the col_to_get, 
    # do same for B, change col names from B to A, and concat

    fighterA_df = fighterA_df[['date', 'Fighter_A', 'A_' + col_to_get]]
    fighterA_df.rename(columns={'A_' + col_to_get: col_to_get, 'Fighter_A': 'fighter'}, inplace=True)
    fighterB_df = fighterB_df[['date', 'Fighter_B', 'B_' + col_to_get]]
    fighterB_df.rename(columns={'B_' + col_to_get: col_to_get, 'Fighter_B': 'fighter'}, inplace=True)
    fighter_data = fighter_data.append(fighterA_df)
    fighter_data = fighter_data.append(fighterB_df)

    # append the dataframes on fighter

    fighter_data = fighter_data.append(fighterA_df)
    fighter_data = fighter_data.append(fighterB_df)

    # get the average

    tot = fighter_data[col_to_get].sum()
    num = fighter_data[col_to_get].count()
    x = tot / num
    return x

In [ ]:
# NEW

for col in the_cols:
    next_fight_df['A_topdown_Avg_' + col ] = next_fight_df.apply(lambda row: get_top_down_averages(row['Fighter_A'], row['date'], col, fights), axis=1)
    next_fight_df['B_topdown_Avg_' + col ] = next_fight_df.apply(lambda row: get_top_down_averages(row['Fighter_B'], row['date'], col, fights), axis=1)

### Add Opponent Stats

In [ ]:
def get_opponent_averages(datafrm, date, col_to_get, fighter):
    col_to_get = 'B_' + col_to_get
    data = datafrm[(datafrm['Fighter_A'] == fighter) | (datafrm['Fighter_B'] == fighter)]
    datey = pd.to_datetime(date)
    data['date'] = pd.to_datetime(data['date'])
    data = data[data['date'] < datey]

    fighter_data = pd.DataFrame()

    fighterA_df = datafrm[datafrm['Fighter_A'] == fighter]
    fighterB_df = datafrm[datafrm['Fighter_B'] == fighter]

    # switch A_ and B_ to B_ and A_ in fighterB_df
    fighterB_df.columns = [n.replace('A_', 'C_') if n.startswith('A_') else n for n in fighterB_df.columns]
    fighterB_df.columns = [n.replace('B_', 'D_') if n.startswith('B_') else n for n in fighterB_df.columns]
    fighterB_df.columns = [n.replace('C_', 'B_') if n.startswith('C_') else n for n in fighterB_df.columns]
    fighterB_df.columns = [n.replace('D_', 'A_') if n.startswith('D_') else n for n in fighterB_df.columns]

    # copy fighter_A and fighter_B columns to new dataframe, then switch fighter_A and fighter_B columns

    fighterB_df.rename(columns={'Fighter_A': 'Fighter_A2', 'Fighter_B': 'Fighter_B2'}, inplace=True)
    fighterB_df.rename(columns={'Fighter_B2': 'Fighter_A', 'Fighter_A2': 'Fighter_B'}, inplace=True)

    fighter_data = fighter_data.append(fighterA_df)
    fighter_data = fighter_data.append(fighterB_df)

    # get sum of col to get
    col_sum = fighter_data[col_to_get].sum()
    # get number of fights
    num_fights = len(fighter_data)
    # get average
    avg = col_sum / num_fights

    return avg


In [ ]:
n=0

for col in the_cols:
    next_fight_df['A_Opp_Avg_' + col] = next_fight_df.apply(lambda row: get_opponent_averages(fights, row['date'], col, row['Fighter_A']), axis=1)
    next_fight_df['B_Opp_Avg_' + col] = next_fight_df.apply(lambda row: get_opponent_averages(fights, row['date'], col, row['Fighter_B']), axis=1)
    n = n+1
    print(f' {col}, #{n} / {len(the_cols)} done.')

In [ ]:
next_fight_df

### Feature: Rolling Career Stat Differentials

#### 1) Mean (Average)

In [ ]:
rolling_mean_cols= [n for n in fights.columns if 'Rolling' in n and 'mean' in n]
# no difs
rolling_mean_cols= [n for n in rolling_mean_cols if 'Dif' not in n]
rolling_mean_cols = pd.DataFrame(rolling_mean_cols)
rolling_mean_cols['nonspecific'] = rolling_mean_cols[0].str[2:]
rmc = rolling_mean_cols['nonspecific'].unique()
rmc

In [ ]:
for col in rmc:
    next_fight_df['Dif_' + col] = next_fight_df['A_' + col] - next_fight_df['B_' + col]

next_fight_df.head(3)

#### 2) Median (Average)

In [ ]:
rolling_median_cols= [n for n in fights.columns if 'Rolling' in n and 'median' in n]
# no difs
rolling_median_cols= [n for n in rolling_median_cols if 'Dif' not in n]
rolling_median_cols = pd.DataFrame(rolling_median_cols)
rolling_median_cols['nonspecific'] = rolling_median_cols[0].str[2:]
rmc = rolling_median_cols['nonspecific'].unique()

In [ ]:
for col in rmc:
    next_fight_df['Dif_' + col] = next_fight_df['A_' + col] - next_fight_df['B_' + col]

next_fight_df.head(3)

#### 3) Standard Deviations

In [ ]:
rolling_std_cols= [n for n in fights.columns if 'Rolling' in n and 'std' in n]
# no difs
rolling_std_cols= [n for n in rolling_std_cols if 'Dif' not in n]

rolling_std_cols = pd.DataFrame(rolling_std_cols)
rolling_std_cols['nonspecific'] = rolling_std_cols[0].str[2:]
rsc = rolling_std_cols['nonspecific'].unique()
rsc

In [ ]:
for col in rsc:
    next_fight_df['Dif_' + col] = next_fight_df['A_' + col] - next_fight_df['B_' + col]

next_fight_df.head(3)

### Feature: UFC Bio Data

In [ ]:
fighter_bios = pd.read_csv('data/final/aggregates/All_Fighter_Bios.csv')
fighter_bios.head(1)

In [ ]:
# if missing place of birth, replace with 'Unknown'
fighter_bios['Place of Birth'].fillna('Unknown', inplace=True)

# if missing fighting style, replace with 'Unknown'
fighter_bios['Fighting style'].fillna('Unknown', inplace=True)

# if missing Trains at, replace with 'Unknown'
fighter_bios['Trains at'].fillna('Unknown', inplace=True)

# if missing status, replace with 'Unknown'
fighter_bios['Status'].fillna('Unknown', inplace=True)

# if missing age, replace with median
fighter_bios['Age'].fillna(fighter_bios['Age'].median(), inplace=True)

# if missing height, replace with median -- thankfully not many missing, as 
# this is a somewhat important feature
fighter_bios['Height'].fillna(fighter_bios['Height'].median(), inplace=True)

# if missing weight, replace with median -- thankfully not many missing
fighter_bios['Weight'].fillna(fighter_bios['Weight'].median(), inplace=True)

In [ ]:
def get_bio_data(fighter, stat):
    try:
        data = fighter_bios[fighter_bios['fighter'] == fighter]
        d = data[stat].values[0]
        d = pd.to_numeric(d)
        return d
    except:
        return nan

In [ ]:
next_fight_df['A_Height'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_A'], 'Height'), axis=1)
next_fight_df['B_Height'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_B'], 'Height'), axis=1)
next_fight_df['Dif_Height'] = next_fight_df['A_Height'] - next_fight_df['B_Height']

next_fight_df['A_Reach'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_A'], 'Reach'), axis=1)
next_fight_df['B_Reach'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_B'], 'Reach'), axis=1)
next_fight_df['Dif_Reach'] = next_fight_df['A_Reach'] - next_fight_df['B_Reach']
next_fight_df['Reach_Dif'] = next_fight_df['A_Reach'] - next_fight_df['B_Reach']

next_fight_df['A_Leg_Reach'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_A'], 'Leg reach'), axis=1)
next_fight_df['B_Leg_Reach'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_B'], 'Leg reach'), axis=1)
next_fight_df['Dif_Leg_Reach'] = next_fight_df['A_Leg_Reach'] - next_fight_df['B_Leg_Reach']
next_fight_df['Leg_Reach_Dif'] = next_fight_df['A_Leg_Reach'] - next_fight_df['B_Leg_Reach']

In [ ]:
# New column indicating if we have the reach data for fighter A
next_fight_df['A_Reach_NA'] = next_fight_df['A_Reach'].isna()

# New column indicating if we have the reach data for fighter B
next_fight_df['B_Reach_NA'] = next_fight_df['B_Reach'].isna()

# New column indicating if we have the reach data for either fighter
next_fight_df['Reach_NA'] = next_fight_df['A_Reach_NA'] | next_fight_df['B_Reach_NA']

# New column indicating if we have leg reach data for fighter A
next_fight_df['A_Leg_Reach_NA'] = next_fight_df['A_Leg_Reach'].isna()

# New column indicating if we have leg reach data for fighter B
next_fight_df['B_Leg_Reach_NA'] = next_fight_df['B_Leg_Reach'].isna()

# New column indicating if we have leg reach data for either fighter
next_fight_df['Leg_Reach_NA'] = next_fight_df['A_Leg_Reach_NA'] | next_fight_df['B_Leg_Reach_NA']

### Feature: Add Weightclass

In [ ]:
all_events = pd.read_csv('data/final/events/All_Events_Fights_and_FightUrls.csv')

In [ ]:
def find_typical_weightclass(fighter):
    try:
        data = all_events[all_events['Fighter1'] == fighter]
        data2 = all_events[all_events['Fighter2'] == fighter]
        data = pd.concat([data, data2])
        d = data['Weight class'].value_counts().index[0]
        return d
    except:
        return nan

In [ ]:
next_fight_df['A_Typical_Weightclass'] = next_fight_df.apply(lambda row: find_typical_weightclass(row['Fighter_A']), axis=1)
next_fight_df['B_Typical_Weightclass'] = next_fight_df.apply(lambda row: find_typical_weightclass(row['Fighter_B']), axis=1)

In [ ]:
# use fight_id to find weightclass of fight
def find_weightclass(fight_id):
    try:
        fight_url = 'http://www.ufcstats.com/fight-details/' + str(fight_id)
        data = all_events[all_events['fight_link'] == fight_url]
        d = data['Weight class'].values[0]
        return d
    except:
        return nan

In [ ]:
next_fight_df['fight_weightclass'] = fights.apply(lambda row: find_weightclass(row['fight_id']), axis=1)

In [ ]:
next_fight_df['A_Fight_in_Typical_Weightclass'] = next_fight_df.apply(lambda row: row['A_Typical_Weightclass'] == row['fight_weightclass'], axis=1)
next_fight_df['B_Fight_in_Typical_Weightclass'] = next_fight_df.apply(lambda row: row['B_Typical_Weightclass'] == row['fight_weightclass'], axis=1)

### Feature: Method

In [ ]:
# split Method into Method and Method Detail
all_events['Method_Primary'] = all_events['Method'].str.split('  ', expand=True)[0]
all_events['Method_Detail'] = all_events['Method'].str.split('  ', expand=True)[1]
all_events.head(3)

In [ ]:
def get_method(fight_id):
    try:
        fight_url = 'http://www.ufcstats.com/fight-details/' + str(fight_id)
        data = all_events[all_events['fight_link'] == fight_url]
        d = data['Method_Primary'].values[0]
        return d
    except:
        return nan

In [ ]:
def get_details(fight_id):
    try:
        fight_url = 'http://www.ufcstats.com/fight-details/' + str(fight_id)
        data = all_events[all_events['fight_link'] == fight_url]
        d = data['Method_Detail'].values[0]
        return d
    except:
        return nan

#### Feature: Round & Time

In [ ]:
def get_round_time(fight_id, round_or_time):
    try: 
        fight_url = 'http://www.ufcstats.com/fight-details/' + str(fight_id)
        data = all_events[all_events['fight_link'] == fight_url]
        if round_or_time == 'round':
            d = data['Round'].values[0]
        elif round_or_time == 'time':
            d = data['Time'].values[0]
        return d
    except:
        return nan

### Feature: Size Ratios

In [ ]:
next_fight_df['A_Ape_Index'] = next_fight_df['A_Reach'] / next_fight_df['A_Height']
next_fight_df['B_Ape_Index'] = next_fight_df['B_Reach'] / next_fight_df['B_Height']

# Leg_Index = Leg_Reach / Height
next_fight_df['A_Leg_Index'] = next_fight_df['A_Leg_Reach'] / next_fight_df['A_Height']
next_fight_df['B_Leg_Index'] = next_fight_df['B_Leg_Reach'] / next_fight_df['B_Height']

# Leg_to_Wing_Index = Leg_Reach / Reach
next_fight_df['A_Leg_to_Wing_Index'] = next_fight_df['A_Leg_Reach'] / next_fight_df['A_Reach']
next_fight_df['B_Leg_to_Wing_Index'] = next_fight_df['B_Leg_Reach'] / next_fight_df['B_Reach']

### Feature: Favorite

In [ ]:
next_fight_df['favorite?'] = np.where(next_fight_df['Fighter_A_Odds'] < next_fight_df['Fighter_B_Odds'], 1, 0)

### Feature: Win/Loss Details

In [ ]:
# add a formatted date column
next_fight_df['datetime'] = pd.to_datetime(next_fight_df['date'])
next_fight_df['date_formatted'] = next_fight_df['datetime'].dt.date

In [ ]:
def get_number_UFC_fights(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    return len(data)


In [ ]:
next_fight_df['A_Total_UFC_Fights'] = fights.apply(lambda row: get_number_UFC_fights(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Total_UFC_Fights'] = fights.apply(lambda row: get_number_UFC_fights(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Total_UFC_Fights'] = next_fight_df['A_Total_UFC_Fights'] - next_fight_df['B_Total_UFC_Fights']

### Wins

In [ ]:
# add number of wins
def get_number_UFC_wins(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    return data['win?'].sum()

In [ ]:
next_fight_df['A_UFC_Wins'] = fights.apply(lambda row: get_number_UFC_wins(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_UFC_Wins'] = fights.apply(lambda row: get_number_UFC_wins(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_UFC_Wins'] = next_fight_df['A_UFC_Wins'] - next_fight_df['B_UFC_Wins']

### Losses

In [ ]:
def get_number_UFC_losses(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    return len(data) - data['win?'].sum()

In [ ]:
next_fight_df['A_UFC_Losses'] = fights.apply(lambda row: get_number_UFC_losses(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_UFC_Losses'] = fights.apply(lambda row: get_number_UFC_losses(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_UFC_Losses'] = next_fight_df['A_UFC_Losses'] - next_fight_df['B_UFC_Losses']

### W/L Percentages

In [ ]:
next_fight_df['A_UFC_Win_Percentage'] = next_fight_df['A_UFC_Wins'] / next_fight_df['A_Total_UFC_Fights']
next_fight_df['B_UFC_Win_Percentage'] = next_fight_df['B_UFC_Wins'] / next_fight_df['B_Total_UFC_Fights']

# add diff
next_fight_df['Dif_UFC_Win_Percentage'] = next_fight_df['A_UFC_Win_Percentage'] - next_fight_df['B_UFC_Win_Percentage']

### Last 5 Win Percentage

In [ ]:
def get_last5_win_percentage(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    data = data.sort_values(by='datetime', ascending=False)
    data = data.head(5)
    return data['win?'].sum() / len(data)

In [ ]:
next_fight_df['A_Last5_Win_Percentage'] = fights.apply(lambda row: get_last5_win_percentage(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Last5_Win_Percentage'] = fights.apply(lambda row: get_last5_win_percentage(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Last5_Win_Percentage'] = next_fight_df['A_Last5_Win_Percentage'] - next_fight_df['B_Last5_Win_Percentage']

### Last 3 Win Percentage

In [ ]:
def get_last3_win_percentage(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    data = data.sort_values(by='datetime', ascending=False)
    data = data.head(3)
    return data['win?'].sum() / len(data)

In [ ]:
next_fight_df['A_Last3_Win_Percentage'] = fights.apply(lambda row: get_last3_win_percentage(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Last3_Win_Percentage'] = fights.apply(lambda row: get_last3_win_percentage(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Last3_Win_Percentage'] = next_fight_df['A_Last3_Win_Percentage'] - next_fight_df['B_Last3_Win_Percentage']

### Win_by/ Loss_by Features

In [ ]:
def get_general_method(fight_method):
    if 'DEC' in fight_method:
        return 'DEC'
    elif 'SUB' in fight_method:
        return 'SUB'
    elif 'KO' in fight_method:
        return 'KO'
    else:
        return 'OTHER'

In [ ]:
def get_win_by_ko_percent(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    wins = data[data['win?'] == 1]
    wins = wins[wins['InFightData__General_Method'] == 'KO']
    return data['win?'].sum() / len(data)

def get_loss_by_ko_percent(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    losses = data[data['win?'] == 0]
    losses = losses[losses['InFightData__General_Method'] == 'KO']
    return data['win?'].sum() / len(data)

In [ ]:
next_fight_df['A_Win_By_KO_Percentage'] = fights.apply(lambda row: get_win_by_ko_percent(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Win_By_KO_Percentage'] = fights.apply(lambda row: get_win_by_ko_percent(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Win_By_KO_Percentage'] = next_fight_df['A_Win_By_KO_Percentage'] - next_fight_df['B_Win_By_KO_Percentage']

In [ ]:
next_fight_df['A_Loss_By_KO_Percentage'] = fights.apply(lambda row: get_loss_by_ko_percent(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Loss_By_KO_Percentage'] = fights.apply(lambda row: get_loss_by_ko_percent(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Loss_By_KO_Percentage'] = next_fight_df['A_Loss_By_KO_Percentage'] - next_fight_df['B_Loss_By_KO_Percentage']

### WL by decision

In [ ]:
def get_win_by_decision_percent(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    wins = data[data['win?'] == 1]
    wins = wins[wins['InFightData__General_Method'] == 'DEC']
    return data['win?'].sum() / len(data)

def get_loss_by_decision_percent(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    losses = data[data['win?'] == 0]
    losses = losses[losses['InFightData__General_Method'] == 'DEC']
    return data['win?'].sum() / len(data)

In [ ]:
next_fight_df['A_Win_By_Decision_Percentage'] = fights.apply(lambda row: get_win_by_decision_percent(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Win_By_Decision_Percentage'] = fights.apply(lambda row: get_win_by_decision_percent(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Win_By_Decision_Percentage'] = next_fight_df['A_Win_By_Decision_Percentage'] - next_fight_df['B_Win_By_Decision_Percentage']

In [ ]:
next_fight_df['A_Loss_By_Decision_Percentage'] = fights.apply(lambda row: get_loss_by_decision_percent(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Loss_By_Decision_Percentage'] = fights.apply(lambda row: get_loss_by_decision_percent(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Loss_By_Decision_Percentage'] = next_fight_df['A_Loss_By_Decision_Percentage'] - next_fight_df['B_Loss_By_Decision_Percentage']

### Career Fight Time

In [ ]:
def get_career_fight_time_seconds(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    return data['InFightData__Total_Fight_Time_Seconds'].sum()

In [ ]:
next_fight_df['A_UFC_Fight_Time_Seconds'] = fights.apply(lambda row: get_career_fight_time_seconds(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_UFC_Fight_Time_Seconds'] = fights.apply(lambda row: get_career_fight_time_seconds(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_UFC_Fight_Time_Seconds'] = next_fight_df['A_UFC_Fight_Time_Seconds'] - next_fight_df['B_UFC_Fight_Time_Seconds']

In [ ]:
next_fight_df

### Stat/Round Features

In [ ]:
topdown_cols = [n for n in next_fight_df.columns.to_list() if 'topdown' in n]
# A TOPDOWN
a_topdown_cols = [n for n in topdown_cols if n.startswith('A_')]
# get rid of any percent columns
a_topdown_cols = [n for n in a_topdown_cols if 'percent' not in n]

# B TOPDOWN
b_topdown_cols = [n for n in topdown_cols if n.startswith('B_')]
# get rid of any percent columns
b_topdown_cols = [n for n in b_topdown_cols if 'percent' not in n]
b_topdown_cols


In [ ]:
len(b_topdown_cols)

In [ ]:
len(a_topdown_cols)

In [ ]:
next_fight_df['A_UFC_Fight_Rounds'] = next_fight_df['A_UFC_Fight_Time_Seconds'] / 300
next_fight_df['B_UFC_Fight_Rounds'] = next_fight_df['B_UFC_Fight_Time_Seconds'] / 300

In [ ]:
# add per-round averages using topdown columns
for col in a_topdown_cols:
    next_fight_df[f'{col}_per_round'] = next_fight_df[col] / next_fight_df['A_UFC_Fight_Rounds']

for col in b_topdown_cols:
    next_fight_df[f'{col}_per_round'] = next_fight_df[col] / next_fight_df['B_UFC_Fight_Rounds']

In [ ]:
next_fight_df

In [ ]:
# get opponent average columns
opp_avg_cols = [n for n in next_fight_df.columns.to_list() if 'Opp_Avg' in n]
# A OPP AVG
a_opp_avg_cols = [n for n in opp_avg_cols if n.startswith('A_')]
# get rid of any percent columns
a_opp_avg_cols = [n for n in a_opp_avg_cols if 'percent' not in n]

# B OPP AVG
b_opp_avg_cols = [n for n in opp_avg_cols if n.startswith('B_')]
# get rid of any percent columns
b_opp_avg_cols = [n for n in b_opp_avg_cols if 'percent' not in n]

b_opp_avg_cols

In [ ]:
# add per-round averages using opponent averages
for col in a_opp_avg_cols:
    next_fight_df[f'{col}_per_round'] = next_fight_df[col] / next_fight_df['A_UFC_Fight_Rounds']

for col in b_opp_avg_cols:
    next_fight_df[f'{col}_per_round'] = next_fight_df[col] / next_fight_df['B_UFC_Fight_Rounds']


#### Per Round Difs

In [ ]:
# add difference in per-round averages

# get per-round topdown columns
topdown_per_round_cols = [n for n in next_fight_df.columns.to_list() if 'topdown' and 'per_round' in n]
# no difs
topdown_per_round_cols = [n for n in topdown_per_round_cols if 'Dif' not in n]

topdown_per_round_cols = pd.DataFrame(topdown_per_round_cols)
topdown_per_round_cols.columns = ['col']
topdown_per_round_cols['non-specific'] = topdown_per_round_cols['col'].str[2:]
# get unique values
topdown_per_round_cols = topdown_per_round_cols['non-specific'].unique().tolist()

In [ ]:
for col in topdown_per_round_cols:
    next_fight_df[f'Dif_{col}'] = next_fight_df[f'A_{col}'] - next_fight_df[f'B_{col}']

In [ ]:
next_fight_df

## Fix NAN and INF

In [ ]:
# replace NANS with 0 for any column starting with A_ or B_
for col in next_fight_df.columns:
    if col.startswith('A_') or col.startswith('B_'):
        next_fight_df[col] = next_fight_df[col].fillna(0)

In [ ]:
# replace inf with 0 for any columns starting with A_ or B_
for col in next_fight_df.columns:
    if col.startswith('A_') or col.startswith('B_'):
        next_fight_df[col] = next_fight_df[col].replace(np.inf, 0)

In [ ]:
# re-calculate all dif columns

# get dif columns
dif_cols = [n for n in next_fight_df.columns.to_list() if 'Dif' in n]

dif_cols

In [ ]:
dif_cols.remove('Reach_Dif')
dif_cols.remove('Leg_Reach_Dif')

In [ ]:
for col in dif_cols:
    next_fight_df[col] = next_fight_df[f'A_{col[4:]}'] - next_fight_df[f'B_{col[4:]}']

In [ ]:
next_fight_df['Reach_Dif'] = next_fight_df['A_Reach'] - next_fight_df['B_Reach']
next_fight_df['Leg_Reach_Dif'] = next_fight_df['A_Leg_Reach'] - next_fight_df['B_Leg_Reach']

In [ ]:
next_fight_df

In [ ]:
# get NAN
missing = next_fight_df.isna().sum()
missing = missing[missing > 0]
missing

In [ ]:
# replace unknown fighter odds with opposite of known fighter odds
next_fight_df['Fighter_A_Odds'] = next_fight_df['Fighter_A_Odds'].fillna(next_fight_df['Fighter_B_Odds'] * -1)
next_fight_df['Fighter_B_Odds'] = next_fight_df['Fighter_B_Odds'].fillna(next_fight_df['Fighter_A_Odds'] * -1)


In [ ]:
# replace unknown fighter change odds with opposite of known fighter odds
next_fight_df['Fighter_A_Odds_Change'] = next_fight_df['Fighter_A_Odds_Change'].fillna(next_fight_df['Fighter_B_Odds_Change'] * -1)
next_fight_df['Fighter_B_Odds_Change'] = next_fight_df['Fighter_B_Odds_Change'].fillna(next_fight_df['Fighter_A_Odds_Change'] * -1)

In [ ]:
# get NAN
missing = next_fight_df.isna().sum()
missing = missing[missing > 0]
missing

In [ ]:
# check difference in columns between next_fight_df and fights
# get columns in next_fight_df that are not in fights
next_fight_cols = next_fight_df.columns.to_list()
fights_cols = fights.columns.to_list()
missing_cols = [n for n in next_fight_cols if n not in fights_cols]
missing_cols

In [ ]:
# get the columns in fights that are not in next_fight_df
missing_cols = [n for n in fights_cols if n not in next_fight_cols]
missing_cols

In [ ]:
the_date = next_fight_df['date'].values[0]
the_date = pd.to_datetime(the_date).date()
# to str
the_date = the_date.strftime('%Y-%m-%d')
the_date

In [ ]:
# save to csv
next_fight_df.to_csv('data/final/next_fights/' + the_date + '.csv', index=False)

In [ ]:
next_fight_df = pd.read_csv('data/final/next_fights/' + the_date + '.csv')


